In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qqq

     |████████████████████████████████| 1.8MB 18.0MB/s 
     |████████████████████████████████| 133kB 52.4MB/s 
     |████████████████████████████████| 163kB 51.6MB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 102kB 13.6MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 


In [3]:
from wandb.keras import WandbCallback
import wandb

In [4]:
# Configure the sweep
sweep_config = {
    'name' : 'Dropout Rate Sweep',
    'method': 'grid',
    'parameters': {
        'dropout1_rate': {
            'values': [0, 0.5]
        },
        'dropout2_rate': {
            'values': [0, 0.5]
        },
        'dropout3_rate': {
            'values': [0, 0.5]
        },
    }
}

In [5]:
WANDB_API_KEY = '002b3552fd78e3559e38c7317ef0d9eefd0c7c6d'
sweep_id = wandb.sweep(sweep_config, project='nn_project_tuning')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 001rfn8c
Sweep URL: https://wandb.ai/burntice/nn_project_tuning/sweeps/001rfn8c


In [6]:
# sweep reference: 
# https://colab.research.google.com/drive/1gKixa6hNUB8qrn1CfHirOfTEQm0qLCSS?usp=sharing

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import tensorflow as tf


# Load dataset as dataframe
base_path = '/content/drive/My Drive/NN Project/'
df = pd.read_csv(base_path + 'aligned_gender.txt', sep='\t')
df['datadir'] = base_path + df['datadir'].astype(str)

# Train test split
train_val_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_val_df, test_size=0.2)

# function to call for hyperparameter sweep
def train():
    # Fix seed
    seed = 7
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # default hyperparameters
    # Use optimal params from first three runs
    defaults = {
        'epochs': 20,
        'batch_size': 64,
        'learning_rate': 5e-4,
        'weight_decay': 0.001,
        'fc1_num_neurons': 512,
        'fc2_num_neurons': 512,
        'fc3_num_neurons': 512,
        'dropout1_rate': 0,
        'dropout2_rate': 0,
        'dropout3_rate': 0,
        'seed': seed,
        'optimizer': 'adam',
        'hidden_activation': 'relu',
        'output_activation': 'sigmoid',
        'loss_function': 'binary_crossentropy',
        'metrics': ['accuracy'],
    }

    wandb.init(
        config=defaults,
        name='Hyperparameter Tuning',
        project='nn_project',
    )
    config = wandb.config

    # Define model
    mobile_net_v2 = tf.keras.applications.MobileNetV2(
        include_top=False,
        pooling='avg',
        weights='imagenet',
        input_shape=(224,224,3),
    )

    fc1 = tf.keras.layers.Dense(
        config.fc1_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    fc2 = tf.keras.layers.Dense(
        config.fc2_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    fc3 = tf.keras.layers.Dense(
        config.fc3_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    bn1 = tf.keras.layers.BatchNormalization()
    bn2 = tf.keras.layers.BatchNormalization()
    bn3 = tf.keras.layers.BatchNormalization()
    bn4 = tf.keras.layers.BatchNormalization()

    dropout1 = tf.keras.layers.Dropout(rate=config.dropout1_rate)
    dropout2 = tf.keras.layers.Dropout(rate=config.dropout2_rate)
    dropout3 = tf.keras.layers.Dropout(rate=config.dropout3_rate)

    model = tf.keras.models.Sequential([
        mobile_net_v2,
        tf.keras.layers.Flatten(),
        bn1,
        fc1,
        dropout1,
        bn2,
        fc2,
        dropout2,
        bn3,
        fc3,
        dropout3,
        bn4,
        tf.keras.layers.Dense(1, activation=config.output_activation),
    ])

    # Compile model 
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
        loss=config.loss_function,
        metrics=config.metrics,
    )

    # Load images into keras image generator 
    datagen_train = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    datagen_val = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    datagen_test = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )

    train_generator = datagen_train.flow_from_dataframe(
        dataframe=train_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    val_generator = datagen_train.flow_from_dataframe(
        dataframe=val_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    test_generator = datagen_test.flow_from_dataframe(
        dataframe=test_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=config.epochs,
        callbacks=[WandbCallback()],
    )

In [8]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: a1cvl8uk with config:
wandb: 	dropout1_rate: 0
wandb: 	dropout2_rate: 0
wandb: 	dropout3_rate: 0
wandb: Currently logged in as: burntice (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


9412608/9406464 [==============================] - 0s 0us/step
Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
122/122 [==============================] - 1913s 16s/step - loss: 2.0849 - accuracy: 0.8328 - val_loss: 1.9192 - val_accuracy: 0.8729
Epoch 2/20
122/122 [==============================] - 146s 1s/step - loss: 1.7094 - accuracy: 0.9334 - val_loss: 2.2678 - val_accuracy: 0.7955
Epoch 3/20
122/122 [==============================] - 148s 1s/step - loss: 1.4589 - accuracy: 0.9608 - val_loss: 1.6378 - val_accuracy: 0.9047
Epoch 4/20
122/122 [==============================] - 152s 1s/step - loss: 1.2332 - accuracy: 0.9733 - val_loss: 1.5048 - val_accuracy: 0.9042
Epoch 5/20
122/122 [==============================] - 152s 1s/step - loss: 1.0465 - accuracy: 0.9755 - val_loss: 4.4511 - val_accuracy: 0.6715
Epoch 6/20
122/122 [==============================] - 151s 1s/step - loss: 0.8673 - accuracy: 0.9798 - val

epoch,19
loss,0.07184
accuracy,0.98283
val_loss,0.49685
val_accuracy,0.91953
_step,19
_runtime,4825
_timestamp,1606039661
best_val_loss,0.35526
best_epoch,18


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▇▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁
accuracy,▁▅▇▇▇▇██████████████
val_loss,▄▄▃▃█▃▄▂▄▃▂▇▃▄▂▂▃▂▁▁
val_accuracy,▇▆██▄▇▆█▄▅▇▁▅▃█▇▄▆██
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██


wandb: Agent Starting Run: 7v2rrb80 with config:
wandb: 	dropout1_rate: 0
wandb: 	dropout2_rate: 0
wandb: 	dropout3_rate: 0.5
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

wandb: Agent Starting Run: 9q3r9vu9 with config:
wandb: 	dropout1_rate: 0.5
wandb: 	dropout2_rate: 0.5
wandb: 	dropout3_rate: 0.5
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
122/122 [==============================] - 152s 1s/step - loss: 2.1815 - accuracy: 0.7718 - val_loss: 2.5720 - val_accuracy: 0.7591
Epoch 2/20
122/122 [==============================] - 153s 1s/step - loss: 1.7593 - accuracy: 0.9148 - val_loss: 2.2112 - val_accuracy: 0.8575
Epoch 3/20
122/122 [==============================] - 150s 1s/step - loss: 1.4747 - accuracy: 0.9590 - val_loss: 1.7442 - val_accuracy: 0.9052
Epoch 4/20
122/122 [==============================] - 152s 1s/step - loss: 1.2619 - accuracy: 0.9719 - val_loss: 1.6414 - val_accuracy: 0.9001
Epoch 5/20
122/122 [==============================] - 150s 1s/step - loss: 1.0624 - accuracy: 0.9800 - val_loss: 1.6506 - val_accuracy: 0.8616
Epoch 6/20
122/122 [==============================] - 147s 1s/step - loss: 0.8984 - accuracy: 0.9804 - val_loss: 2.7546 - val_accuracy: 0.7140
Epoch 7/20
122/122 [========

epoch,19
loss,0.07909
accuracy,0.98719
val_loss,0.77214
val_accuracy,0.85956
_step,19
_runtime,3011
_timestamp,1606059981
best_val_loss,0.48494
best_epoch,11


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▇▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁
accuracy,▁▆▇▇████████████████
val_loss,▇▆▅▅▅█▂▂▂▁▁▁▂▁▃▁▁▂▁▂
val_accuracy,▃▆▇▇▆▁██▇█▇█▇▇▅▇▆▇█▆
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▃▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▃▃▃▄▄▄▅▅▅▆▆▇▇▇██


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
